In [ ]:
#查看零件数据集
#搞懂格式 -》获取part label
#放到场景里面

In [ ]:
import trimesh
import numpy as np
from shapely.geometry import LineString
import os
import random
%pylab inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
def generate_pth_cls(category,id):
    mesh_base_dir = '/home/lidosan/Datasets/PartNet/{}/models'.format(category)
    return os.path.join(mesh_base_dir,id+'.obj')


In [ ]:
import trimesh
import numpy as np
import random

# Assuming generate_pth_cls is defined and works as expected
def generate_pth_cls(category,id):
    mesh_base_dir = '/home/lidosan/Datasets/PartNet/{}/models'.format(category)
    # mesh_base_dir = 'scene1/{}_id'.format(category)
    return os.path.join(mesh_base_dir,id+'.obj')

# Assuming generate_pth_cls is defined and works as expected
def generate_pth_scene(category,id):
    scene = 'scene1'
    # mesh_base_dir = '/home/lidosan/Datasets/PartNet/{}/models'.format(category)
    obj_dir = f"{scene}/{category}_{id}.obj"
    return obj_dir

# Function to scale the mesh to fit within the given range
def scale_mesh(mesh, min_val, max_val):
    scale_factor = (max_val - min_val) / max(mesh.extents)
    mesh.apply_scale(scale_factor)
    return mesh

# Function to align the mesh on the ground (z=0)
def align_mesh_to_ground(mesh):
    min_z = mesh.bounds[0][2]
    mesh.apply_translation([0, 0, -min_z])
    return mesh

def rotate_mesh_x_90(mesh):
    rotation_matrix = trimesh.transformations.rotation_matrix(np.radians(90), [1, 0, 0])
    mesh.apply_transform(rotation_matrix)
    return mesh

# Function to rotate the mesh around the z-axis
def rotate_mesh_z(mesh, angle_degrees):
    rotation_matrix = trimesh.transformations.rotation_matrix(np.radians(angle_degrees), [0, 0, 1])
    mesh.apply_transform(rotation_matrix)
    return mesh

# Generate paths for each object
bed_path = generate_pth_scene('Bed', '10033')
chair_path = generate_pth_scene('Chair', '43634')
table_path = generate_pth_scene('Table', '27996')
trashcan_path = generate_pth_scene('TrashCan', '12100')
lamp_path = generate_pth_scene('Lamp','15052')
laptop_path = generate_pth_scene('Laptop','10222')


# List of OBJ file paths to combine and their respective scales
obj_files = [
    (bed_path, (-3, 3)),
    (chair_path, (-1.5,1.5)),
    (table_path, (-2, 2)),
    (trashcan_path, (-0.5, 0.5)),
    (lamp_path, (-1, 1)),
    (laptop_path, (-0.5, 0.5)),
]

# Initialize an empty list to hold the transformed meshes
meshes = []

# Load, scale, and align each OBJ file
for obj_file, scale_range in obj_files:
    mesh = trimesh.load(obj_file)
    rotate_mesh = rotate_mesh_x_90(mesh)
    scaled_mesh = scale_mesh(rotate_mesh, scale_range[0], scale_range[1])
    aligned_mesh = align_mesh_to_ground(scaled_mesh)
    meshes.append(aligned_mesh)

# Randomly arrange the meshes on the x and y axes
room_size_x = 10  # Define the size of the room in x direction
room_size_y = 10  # Define the size of the room in y direction


# Combine all meshes into a single mesh

# Export the combined mesh to a new OBJ file
# output_file = 'combined_mesh.obj'
# combined_mesh.export(output_file)

# print(f'Combined mesh saved to {output_file}')


# Random Layout

In [ ]:
# Randomly arrange the meshes on the x and y axes without overlapping
room_size_x = 10  # Define the size of the room in x direction
room_size_y = 10  # Define the size of the room in y direction
placed_meshes = []
positions= []
for mesh in meshes:
    collision = True
    while collision:
        position = [random.uniform(-room_size_x / 2, room_size_x / 2),
                    random.uniform(-room_size_y / 2, room_size_y / 2),
                    0]
        mesh.apply_translation(position)
        collision = False
        for placed_mesh in placed_meshes:
            if mesh.bounds[1][0] > placed_mesh.bounds[0][0] and mesh.bounds[0][0] < placed_mesh.bounds[1][0] and \
               mesh.bounds[1][1] > placed_mesh.bounds[0][1] and mesh.bounds[0][1] < placed_mesh.bounds[1][1]:
                collision = True
                mesh.apply_translation([-position[0], -position[1], 0])  # Move back to original position
                break

    placed_meshes.append(mesh)
combined_mesh = trimesh.util.concatenate(meshes)


In [ ]:

# Randomly arrange the meshes on the x and y axes without overlapping
room_size_x = 10  # Define the size of the room in x direction
room_size_y = 10  # Define the size of the room in y direction
placed_meshes = []
positions= []
for mesh in meshes:
    collision = True
    while collision:
        position = [random.uniform(-room_size_x / 2, room_size_x / 2),
                    random.uniform(-room_size_y / 2, room_size_y / 2),
                    0]
        mesh.apply_translation(position)
        collision = False
        for placed_mesh in placed_meshes:
            if mesh.bounds[1][0] > placed_mesh.bounds[0][0] and mesh.bounds[0][0] < placed_mesh.bounds[1][0] and \
               mesh.bounds[1][1] > placed_mesh.bounds[0][1] and mesh.bounds[0][1] < placed_mesh.bounds[1][1]:
                collision = True
                mesh.apply_translation([-position[0], -position[1], 0])  # Move back to original position
                break

    placed_meshes.append(mesh)
combined_mesh = trimesh.util.concatenate(meshes)


# Custom layout

In [ ]:
# Function to rotate the mesh around its center
def rotate_mesh_around_center(mesh, angle_degrees, axis='z'):
    # Calculate the center of the mesh
    center = mesh.centroid
    # Create a translation matrix to move the mesh to the origin
    translation_to_origin = trimesh.transformations.translation_matrix(-center)
    # Create a rotation matrix
    if axis == 'z':
        rotation_matrix = trimesh.transformations.rotation_matrix(np.radians(angle_degrees), [0, 0, 1])
    elif axis == 'y':
        rotation_matrix = trimesh.transformations.rotation_matrix(np.radians(angle_degrees), [0, 1, 0])
    elif axis == 'x':
        rotation_matrix = trimesh.transformations.rotation_matrix(np.radians(angle_degrees), [1, 0, 0])
    else:
        raise ValueError("Invalid axis. Choose from 'x', 'y', or 'z'.")
    # Create a translation matrix to move the mesh back to its original position
    translation_back = trimesh.transformations.translation_matrix(center)
    # Combine the transformations
    transform_matrix = translation_back @ rotation_matrix @ translation_to_origin
    # Apply the transformation
    mesh.apply_transform(transform_matrix)
    return mesh

In [ ]:
# # Load, scale, rotate, and align each OBJ file
# meshes = []
# for obj_file, scale_range in obj_files:
#     mesh = trimesh.load(obj_file)
#     scaled_mesh = scale_mesh(mesh, scale_range[0], scale_range[1])
#     rotated_mesh = rotate_mesh_x_90(scaled_mesh)
#     aligned_mesh = align_mesh_to_ground(rotated_mesh)
#     meshes.append(aligned_mesh)

# Extract individual meshes for easier manipulation
bed_mesh, chair_mesh, table_mesh, trashcan_mesh,laptop_mesh,lamp_mesh = meshes

# Define positions based on the described layout
table_position = [0, 0, 0]
chair_position = [-0.5, -1.5, 0]  # Chair close to table, head to table
trashcan_position = [1, -1, 0]  # Trash bin close to table
bed_position = [4, 0, 0]  # Bed on the right side of the table, chair, and trashcan

# Calculate the height of the table to place objects on top of it
table_height = table_mesh.bounds[1][2]

# Position for laptop and lamp on the table
laptop_position = [0.5, 0, table_height]
lamp_position = [-1, 0, table_height]



# Apply translations to place each mesh in the correct position
table_mesh.apply_translation(table_position)
chair_mesh.apply_translation(chair_position)
chair_mesh = rotate_mesh_around_center(chair_mesh,180)
trashcan_mesh.apply_translation(trashcan_position)
bed_mesh.apply_translation(bed_position)
laptop_mesh.apply_translation(laptop_position)
lamp_mesh.apply_translation(lamp_position)

 
# Combine all meshes into a single mesh
combined_mesh = trimesh.util.concatenate([table_mesh, chair_mesh, trashcan_mesh, bed_mesh,laptop_mesh,lamp_mesh])


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(combined_mesh.vertices[:, 0], combined_mesh.vertices[:,1], triangles=combined_mesh.faces, Z=combined_mesh.vertices[:,2]) 

plt.show()

In [ ]:
combined_mesh.vertices[: ]

In [ ]:
# # Randomly arrange the meshes on the x and y axes without overlapping
# room_size_x = 10  # Define the size of the room in x direction
# room_size_y = 10  # Define the size of the room in y direction
# placed_meshes = []
# positions= []
# for mesh in meshes:
#     collision = True
#     while collision:
#         position = [random.uniform(-room_size_x / 2, room_size_x / 2),
#                     random.uniform(-room_size_y / 2, room_size_y / 2),
#                     0]
#         mesh.apply_translation(position)
#         collision = False
#         for placed_mesh in placed_meshes:
#             if mesh.bounds[1][0] > placed_mesh.bounds[0][0] and mesh.bounds[0][0] < placed_mesh.bounds[1][0] and \
#                mesh.bounds[1][1] > placed_mesh.bounds[0][1] and mesh.bounds[0][1] < placed_mesh.bounds[1][1]:
#                 collision = True
#                 mesh.apply_translation([-position[0], -position[1], 0])  # Move back to original position
#                 break

#     placed_meshes.append(mesh)
# combined_mesh = trimesh.util.concatenate(meshes)


In [ ]:
combined_mesh.show()
# combined_mesh.export('scene1/scene1.obj')


In [ ]:
combined_mesh

Pyviz3d

In [ ]:
# Visualization with pyviz3d
v = viz.Visualizer(position=[5, 5, 1])

# Add each mesh to the visualizer
# v.add_mesh('Bed', path=bed_path, translation=np.array(bed_position))
v.add_mesh('Chair', path=chair_path, translation=np.array(chair_position), rotation=viz.euler_to_quaternion(np.pi/2, 0, np.pi),scale=np.array([1.5,1.5,1.5]))
v.add_mesh('Table', path=table_path, translation=np.array(table_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([2,2,2]))
v.add_mesh('Trashcan', path=trashcan_path, translation=np.array(trashcan_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([0.5,0.5,0.5]))
v.add_mesh('Laptop', path=laptop_path, translation=np.array(laptop_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([0.5,0.5,0.5]))
v.add_mesh('Lamp', path=lamp_path, translation=np.array(lamp_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([0.5,0.5,0.5]))

# Optionally save the visualization
# blender_args = {'output_prefix': './', 'executable_path': '/Applications/Blender.app/Contents/MacOS/Blender'}
# v.save('example_scene', blender_args=blender_args)

# Display the visualization
v.save('exmaple_scene1')

In [ ]:
# Visualization with pyviz3d
v = viz.Visualizer(position=[5, 5, 1])

# Add each mesh to the visualizer
# v.add_mesh('Bed', path=bed_path, translation=np.array(bed_position))
v.add_mesh('Chair', path=chair_path, translation=np.array(chair_position), rotation=viz.euler_to_quaternion(np.pi/2, 0, np.pi),scale=np.array([1.5,1.5,1.5]))
v.add_mesh('Table', path=table_path, translation=np.array(table_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([2,2,2]))
v.add_mesh('Trashcan', path=trashcan_path, translation=np.array(trashcan_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([0.5,0.5,0.5]))
v.add_mesh('Laptop', path=laptop_path, translation=np.array(laptop_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([0.5,0.5,0.5]))
v.add_mesh('Lamp', path=lamp_path, translation=np.array(lamp_position),rotation=viz.euler_to_quaternion(0, 0, np.pi),scale=np.array([0.5,0.5,0.5]))

# Optionally save the visualization
# blender_args = {'output_prefix': './', 'executable_path': '/Applications/Blender.app/Contents/MacOS/Blender'}
# v.save('example_scene', blender_args=blender_args)

# Display the visualization
v.save('exmaple_scene1')